In [ ]:
!pip -q install kaggle groq pandas numpy tqdm tabulate

^C



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\fynd-task-1\\.venv\\Lib\\site-packages\\groq\\types\\file_list_response.py'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json, os
from pathlib import Path

username = "purva0017"
key = "KGAT_1037fdb2515cfa882c88005616c2f26e" # enter your kaggle key here

kaggle_dict = {"username": username, "key": key}

# Windows Kaggle location: C:\Users\<user>\.kaggle\kaggle.json
kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)

kaggle_path = kaggle_dir / "kaggle.json"
with open(kaggle_path, "w") as f:
    json.dump(kaggle_dict, f)

# Kaggle requires permissions like 600 on Linux/macOS.
# On Windows chmod doesn't behave the same, but it's fine to keep:
try:
    os.chmod(kaggle_path, 0o600)
except Exception as e:
    print("chmod skipped on Windows:", e)

print(f"kaggle.json created at: {kaggle_path}")


kaggle.json created at: C:\Users\Purva A Patel\.kaggle\kaggle.json


In [4]:
!pip -q install kaggle
!kaggle datasets list -s yelp


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ref                                                       title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
yelp-dataset/yelp-dataset                                 Yelp Dataset                                      4374983563  2022-03-17 22:59:01.257000         153455       1765  0.75             
ilhamfp31/yelp-review-dataset                             Yelp Review Sentiment Dataset                      169591198  2020-01-29 06:22:07.300000           4220         36  0.7058824        
farukalam/yelp-restaurant-reviews                         Yelp Restaurant Reviews                              3781843  2022-12-31 12:03:48.813000           1944          9  1                
omkarsabnis/yelp-reviews-dataset        

In [ ]:
!kaggle datasets download -d omkarsabnis/yelp-reviews-dataset
!unzip -q yelp-reviews-dataset.zip -d yelp_data
!ls yelp_data

Dataset URL: https://www.kaggle.com/datasets/omkarsabnis/yelp-reviews-dataset
License(s): other




  0%|          | 0.00/3.49M [00:00<?, ?B/s]
 29%|██▊       | 1.00M/3.49M [00:01<00:03, 680kB/s]
 57%|█████▋    | 2.00M/3.49M [00:01<00:01, 1.37MB/s]
 86%|████████▌ | 3.00M/3.49M [00:02<00:00, 1.77MB/s]
100%|██████████| 3.49M/3.49M [00:02<00:00, 1.63MB/s]


yelp.csv


In [8]:
import pandas as pd
import numpy as np

# Adjust filename based on what appears in yelp_data folder
df = pd.read_csv("yelp_data/yelp.csv")  # change if file name differs
df = df.dropna(subset=["text", "stars"])  # ensure clean

# sample for evaluation
sample_df = df.sample(n=200, random_state=42).reset_index(drop=True)
sample_df = sample_df[["text", "stars"]]
sample_df.head()

,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [9]:
import os
from getpass import getpass
from groq import Groq

# 4 keys: one for each prompt run + one for consistency
os.environ["GROQ_API_KEY_V1"] = getpass("Enter Groq API Key for V1: ")
os.environ["GROQ_API_KEY_V2"] = getpass("Enter Groq API Key for V2: ")
os.environ["GROQ_API_KEY_V3"] = getpass("Enter Groq API Key for V3: ")
os.environ["GROQ_API_KEY_CONSISTENCY"] = getpass("Enter Groq API Key for CONSISTENCY: ")

client_v1 = Groq(api_key=os.environ["GROQ_API_KEY_V1"])
client_v2 = Groq(api_key=os.environ["GROQ_API_KEY_V2"])
client_v3 = Groq(api_key=os.environ["GROQ_API_KEY_V3"])
client_cons = Groq(api_key=os.environ["GROQ_API_KEY_CONSISTENCY"])

In [10]:
def prompt_v1(review):
    return f"""
You are a Yelp review star rating classifier.

Decide an integer rating from 1 to 5 based on the review sentiment.

STRICT OUTPUT RULES:
- Output ONLY a valid JSON object.
- Do NOT output markdown, backticks, code fences, headings, or any extra text.
- Use EXACTLY these keys: "predicted_stars", "explanation"
- "predicted_stars" must be an integer in [1,2,3,4,5].
- "explanation" must be <= 15 words (one sentence).
- If unsure, choose the closest rating. Never output null.

Rating guide:
1 = very negative, terrible experience
2 = mostly negative
3 = mixed/neutral
4 = mostly positive with minor issues
5 = extremely positive, excellent

Review:
{review}

Return ONLY JSON:
""".strip()


def prompt_v2(review):
    return f"""
You are an expert Yelp review rating model.

Your job: predict the Yelp star rating (1-5) that best matches the reviewer’s overall satisfaction.

Decision factors (in priority order):
1) Overall satisfaction (recommend or not)
2) Severity of complaints (major vs minor)
3) Service quality
4) Product/food quality
5) Price/value

Rating rubric:
5 = Excellent: strongly recommended, no meaningful complaints
4 = Good: mostly positive, minor complaint(s)
3 = Average: mixed feelings or neutral experience
2 = Poor: mostly negative, multiple problems, not recommended
1 = Terrible: extremely negative, unacceptable experience

STRICT OUTPUT RULES:
- Output ONLY a valid JSON object.
- No markdown, no backticks, no extra words.
- JSON must match this schema exactly:
  {{
    "predicted_stars": <integer 1-5>,
    "explanation": "<one sentence, <= 15 words>"
  }}
- If uncertain between two ratings, pick the lower one.
- Never output null.

Review:
{review}

Return ONLY JSON:
""".strip()


def prompt_v3(review):
    return f"""
You are a Yelp review star rating classifier.

STRICT OUTPUT RULES:
- Output ONLY a valid JSON object.
- Do NOT output markdown/backticks/extra text.
- Use EXACTLY these keys: "predicted_stars", "explanation"
- "predicted_stars" must be integer 1-5.
- "explanation" must be <= 15 words.
- If unsure, choose the closest rating (prefer lower if borderline).

Rubric:
5 = Excellent, very satisfied, would strongly recommend
4 = Good, satisfied, minor issues
3 = Mixed/average, neutral or balanced positives and negatives
2 = Negative, dissatisfied, several problems
1 = Very negative, unacceptable, angry tone

Examples:
Review: "Absolutely amazing! Delicious food, friendly staff, and fast service."
Output: {{"predicted_stars": 5, "explanation": "Strongly positive experience with no meaningful complaints."}}

Review: "Good food but service was slow and the place was noisy."
Output: {{"predicted_stars": 4, "explanation": "Mostly positive experience with a minor service issue."}}

Review: "It was okay. Nothing special. Might try again but not excited."
Output: {{"predicted_stars": 3, "explanation": "Neutral review with no strong positives or negatives."}}

Review: "Bad service and overpriced. Food arrived late and was bland."
Output: {{"predicted_stars": 2, "explanation": "Mostly negative experience with multiple issues."}}

Review: "Worst experience ever. Staff was rude and food was inedible."
Output: {{"predicted_stars": 1, "explanation": "Extremely negative experience with unacceptable service and quality."}}

Now classify this review:

Review:
{review}

Return ONLY JSON:
""".strip()


In [11]:
import json
import re

def extract_json(text):
    """
    Extract JSON object from a model response safely.
    Returns dict or None
    """
    try:
        return json.loads(text)
    except:
        pass

    # fallback: find first {...} block
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None


def validate_output(data):
    """
    Validate JSON structure and constraints.
    """
    if not isinstance(data, dict):
        return False

    if "predicted_stars" not in data or "explanation" not in data:
        return False

    if not isinstance(data["predicted_stars"], int):
        return False

    if data["predicted_stars"] < 1 or data["predicted_stars"] > 5:
        return False

    if not isinstance(data["explanation"], str) or len(data["explanation"].strip()) == 0:
        return False

    return True


In [12]:
from tqdm import tqdm
import time

MODEL_NAME = "llama-3.3-70b-versatile"

def groq_predict(prompt, client):
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=80
    )
    return resp.choices[0].message.content


In [13]:
from tqdm import tqdm
import time
import pandas as pd
import os

def run_prompt_eval(sample_df, prompt_fn, name, client, checkpoint_path):
    """
    Runs prompt evaluation with checkpointing.
    If checkpoint exists, resumes from last completed review_id.
    """

    # Resume if file exists
    if os.path.exists(checkpoint_path):
        existing = pd.read_csv(checkpoint_path)
        done_ids = set(existing["review_id"].astype(int).tolist())
        print(f"[{name}] Resuming from checkpoint. Already done: {len(done_ids)} rows.")
    else:
        existing = None
        done_ids = set()
        print(f"[{name}] No checkpoint found. Starting fresh.")

    rows_to_run = [i for i in range(len(sample_df)) if i not in done_ids]

    out_rows = []  # buffer

    for i in tqdm(rows_to_run, desc=f"Running {name}"):
        review = sample_df.loc[i, "text"]
        actual = int(sample_df.loc[i, "stars"])

        raw = groq_predict(prompt_fn(review), client=client)
        parsed = extract_json(raw)

        valid = validate_output(parsed)
        predicted = parsed["predicted_stars"] if valid else None
        explanation = parsed["explanation"] if valid else None

        out_rows.append({
            "review_id": i,
            "actual_stars": actual,
            "predicted_stars": predicted,
            "valid_json": valid,
            "explanation": explanation,
            "raw_output": raw
        })

        # Append every row immediately (no data loss)
        pd.DataFrame(out_rows).to_csv(
            checkpoint_path,
            mode="a",
            header=not os.path.exists(checkpoint_path),
            index=False
        )
        out_rows = []

        time.sleep(0.2)

    # Load full results
    return pd.read_csv(checkpoint_path)

In [14]:
res_v1 = run_prompt_eval(sample_df, prompt_v1, "Prompt V1", client_v1, "checkpoint_v1.csv")
res_v2 = run_prompt_eval(sample_df, prompt_v2, "Prompt V2", client_v2, "checkpoint_v2.csv")
res_v3 = run_prompt_eval(sample_df, prompt_v3, "Prompt V3", client_v3, "checkpoint_v3.csv")

[Prompt V1] No checkpoint found. Starting fresh.


Running Prompt V1: 100%|██████████| 200/200 [10:32<00:00,  3.16s/it]


[Prompt V2] No checkpoint found. Starting fresh.


Running Prompt V2: 100%|██████████| 200/200 [07:40<00:00,  2.30s/it]


[Prompt V3] No checkpoint found. Starting fresh.


Running Prompt V3:  85%|████████▌ | 170/200 [07:22<01:18,  2.61s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kf94yq8pe8stpatzht7dy0cc` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99972, Requested 479. Please try again in 6m29.664s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [16]:
from getpass import getpass
from groq import Groq
import os

# New key ONLY for continuing V3
os.environ["GROQ_API_KEY_V3B"] = getpass("Enter NEW Groq API Key for V3 continuation: ")
client_v3b = Groq(api_key=os.environ["GROQ_API_KEY_V3B"])

In [17]:
res_v3 = run_prompt_eval(sample_df, prompt_v3, "Prompt V3", client_v3b, "checkpoint_v3.csv")

[Prompt V3] Resuming from checkpoint. Already done: 171 rows.


Running Prompt V3: 100%|██████████| 29/29 [00:24<00:00,  1.17it/s]


In [22]:
# Rename columns so they don't clash
res_v1_ren = res_v1.rename(columns={
    "predicted_stars": "v1_predicted_stars",
    "valid_json": "v1_valid_json",
    "explanation": "v1_explanation",
    "raw_output": "v1_raw_output"
})

res_v2_ren = res_v2.rename(columns={
    "predicted_stars": "v2_predicted_stars",
    "valid_json": "v2_valid_json",
    "explanation": "v2_explanation",
    "raw_output": "v2_raw_output"
})

res_v3_ren = res_v3.rename(columns={
    "predicted_stars": "v3_predicted_stars",
    "valid_json": "v3_valid_json",
    "explanation": "v3_explanation",
    "raw_output": "v3_raw_output"
})

# Merge on review_id + actual_stars
merged = res_v1_ren.merge(
    res_v2_ren[["review_id", "v2_predicted_stars", "v2_valid_json", "v2_explanation", "v2_raw_output"]],
    on="review_id",
    how="outer"
).merge(
    res_v3_ren[["review_id", "v3_predicted_stars", "v3_valid_json", "v3_explanation", "v3_raw_output"]],
    on="review_id",
    how="outer"
)

# Ensure actual_stars exists
if "actual_stars" not in merged.columns:
    merged["actual_stars"] = sample_df["stars"].astype(int)

# Save combined
merged.to_csv("all_prompt_outputs.csv", index=False)
print("Saved combined CSV: all_prompt_outputs.csv ✅")

Saved combined CSV: all_prompt_outputs.csv ✅


In [18]:
def compute_metrics(df):
    json_validity = df["valid_json"].mean() * 100
    accuracy = (df["actual_stars"] == df["predicted_stars"]).mean() * 100
    return accuracy, json_validity

In [ ]:
import os
import pandas as pd
import time
from tqdm import tqdm

def consistency_test_checkpointed(sample_df, prompt_fn, client, n=20, repeats=3, checkpoint_path="consistency_checkpoint.csv"):
    """
    Checkpointed consistency test:
    - Saves each (repeat, review_id) prediction immediately to CSV
    - Resumes without recomputing already-done repeat rows
    - Returns consistency percentage
    """

    subset = sample_df.sample(n=n, random_state=7).reset_index(drop=True)

    # Load checkpoint if exists
    if os.path.exists(checkpoint_path):
        existing = pd.read_csv(checkpoint_path)
        done = set(zip(existing["repeat"].astype(int), existing["review_id"].astype(int)))
        print(f"[Consistency] Resuming. Done predictions: {len(done)}")
    else:
        existing = None
        done = set()
        print("[Consistency] No checkpoint found. Starting fresh.")

    # Run and checkpoint each prediction
    for r in range(repeats):
        for i in tqdm(range(len(subset)), desc=f"Consistency run {r+1}/{repeats}"):
            if (r, i) in done:
                continue

            raw = groq_predict(prompt_fn(subset.loc[i, "text"]), client=client)
            parsed = extract_json(raw)

            if validate_output(parsed):
                pred = parsed["predicted_stars"]
                valid = True
            else:
                pred = None
                valid = False

            # Save immediately (append mode)
            pd.DataFrame([{
                "repeat": r,
                "review_id": i,
                "actual_stars": int(subset.loc[i, "stars"]),
                "predicted_stars": pred,
                "valid_json": valid,
                "raw_output": raw
            }]).to_csv(
                checkpoint_path,
                mode="a",
                header=not os.path.exists(checkpoint_path),
                index=False
            )

            time.sleep(0.2)

    # Load all and compute consistency
    dfc = pd.read_csv(checkpoint_path)

    # Pivot into [review_id x repeats]
    pivot = dfc.pivot_table(index="review_id", columns="repeat", values="predicted_stars", aggfunc="first")

    consistent = 0
    valid_total = 0

    for i in pivot.index:
        row = pivot.loc[i].tolist()
        # must have all repeats present and non-null
        if any(pd.isna(x) for x in row):
            continue
        valid_total += 1
        if len(set(row)) == 1:
            consistent += 1

    if valid_total == 0:
        return 0.0

    return (consistent / valid_total) * 100

In [20]:
cons_v1 = consistency_test_checkpointed(sample_df, prompt_v1, client=client_cons,
                                       checkpoint_path="consistency_v1.csv")

cons_v2 = consistency_test_checkpointed(sample_df, prompt_v2, client=client_cons,
                                       checkpoint_path="consistency_v2.csv")

cons_v3 = consistency_test_checkpointed(sample_df, prompt_v3, client=client_cons,
                                       checkpoint_path="consistency_v3.csv")


[Consistency] No checkpoint found. Starting fresh.


Consistency run 3/3: 100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


[Consistency] No checkpoint found. Starting fresh.


Consistency run 3/3: 100%|██████████| 20/20 [00:52<00:00,  2.62s/it]


[Consistency] No checkpoint found. Starting fresh.


Consistency run 3/3: 100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


In [21]:
from tabulate import tabulate

acc1, val1 = compute_metrics(res_v1)
acc2, val2 = compute_metrics(res_v2)
acc3, val3 = compute_metrics(res_v3)

comparison = pd.DataFrame([
    ["Prompt V1 (Strict JSON + Short Rubric)", acc1, val1, cons_v1],
    ["Prompt V2 (Detailed Rubric + Locked Rules)", acc2, val2, cons_v2],
    ["Prompt V3 (Few-shot + Rubric + Strict JSON)", acc3, val3, cons_v3]
], columns=["Approach", "Accuracy (%)", "JSON Validity (%)", "Consistency (%)"])

print(tabulate(comparison, headers="keys", tablefmt="github", showindex=False))


| Approach                                    |   Accuracy (%) |   JSON Validity (%) |   Consistency (%) |
|---------------------------------------------|----------------|---------------------|-------------------|
| Prompt V1 (Strict JSON + Short Rubric)      |           56.5 |                 100 |               100 |
| Prompt V2 (Detailed Rubric + Locked Rules)  |           63   |                 100 |               100 |
| Prompt V3 (Few-shot + Rubric + Strict JSON) |           58   |                 100 |               100 |
